# Cuadrados mínimos no-lineal paso a paso

Calibración del detector de superficie del Observatorio Pierre Auger

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

## Datos

Esta demo usa datos abiertos disponibles en https://doi.org/10.5281/zenodo.4487612.

In [ ]:
data = pd.read_csv("cuadrados_minimos_nolineal.csv")
xdata = data["energy"]
ydata = data["shower_size"]
yerror = data["shower_size_error"]
print(data)

## Modelo

In [ ]:
def fit_model(energy, par):
    size_0 = 30          # Reference shower size
    energy_0 = par[0]
    power_law_index = par[1]
    return size_0 * np.power(energy / energy_0, 1 / power_law_index)

## Función de costo

In [ ]:
def fit_cost(x, y, ysigma, par):
        mu = fit_model(x, par)
        residuals = (y - mu) / ysigma
        chi_squares = residuals**2
        cost = chi_squares.sum()
        return cost

In [ ]:
J = lambda par: fit_cost(xdata, ydata, yerror, par)

## Minimización

In [ ]:
help(scipy.optimize.minimize)

Changing the tolerance from the default value $1e-7$ to force the convergence. This is necessary because there are many data. The tolerance must be higher than the value reported for the gradient (jacobian) at the minimum or  (jac variable in the output). 

In [ ]:
res = scipy.optimize.minimize(J, x0=(7, 1), tol=1e-4)
res

## Estimadores

In [ ]:
par_est = res.x
par_est

## Errores

In [ ]:
cova = 2*res.hess_inv
error = np.sqrt( np.diagonal(cova) )
rho = cova[0][1]/(error[0]*error[1])
print(f"a = {par_est[0]:.3f} ± {error[0]:.3f}")
print(f"b = {par_est[1]:.3f} ± {error[1]:.3f}")
print(f"ρ = {rho:.2f}")

## Bondad del ajuste

In [ ]:
cost_min = res.fun
ndof = len(xdata) - len(par_est)
pvalor = scipy.stats.chi2.sf(cost_min, ndof)
print(f"χ²min = {cost_min:.2f}")
print(f"ndof = {ndof}")
print(f"pvalor = {pvalor*100:.2}%")

## Plot del ajuste

In [ ]:
xfit = np.linspace( xdata.min(), xdata.max(), 256)
mu_est = fit_model(xfit, par_est)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Energy (EeV)")
ax.set_ylabel("Shower size (VEM)")
ax.errorbar(xdata, ydata, yerror,ls='none', marker='o', ms=4, label="Datos")
ax.plot(xfit, mu_est, label="Ajuste")
plt.legend()
plt.show()